In [47]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.webdriver import WebDriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import time

In [66]:
# Read the locally saved HTML file
with open("fast_nu_isb.html", "r", encoding="utf-8") as html_file:
    html_content = html_file.read()
# html_content

In [67]:
# Create a BeautifulSoup object
soup = BeautifulSoup(html_content, "html.parser")
#print(soup.prettify())

In [89]:
chrome_path = "/Users/shahbanowaqar/Downloads/chromedriver_mac_arm64 (1)/chromedriver" 
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")

driver: WebDriver = webdriver.Chrome(service=Service(chrome_path), options=chrome_options)
    
wait = WebDriverWait(driver, 10)

# Iterate for each faculty member

In [90]:
id_=[]
name=[]
designation=[]
phdSupervisor=[]
email=[]
dept=[]
ext=[]
img=[]
highesteducation=[]

In [91]:
f = soup.findAll('div', class_='our-team')

for i in range(len(f)):
    faculty = f[i]
    
    ############################
    facultyId = faculty.find('img')['id'][-4:]
    facultyName = faculty.find('h3', class_='title').text.strip()
    facultyDesignation = faculty.find('span', class_='post').text.strip()
    facultyEmail = faculty.find('p').text
    facultyDepartment = faculty.find_previous('div', class_='animated fadeInLeft').text
    
    #Faculty link
    facultyLink = faculty.find('ul', class_='social').find_next('a')['href']
    
    re = requests.get(facultyLink)
    soup2 = BeautifulSoup(re.content, 'html.parser')
        
    facultyExtension = None
    p_element = soup2.findAll('p', style = 'line-height: 20px;margin-bottom: 0px;')
    if p_element:
        for i in range(len(p_element)):
            ext_link = p_element[i].find('span')
            if ext_link is not None:
                facultyExtension = ext_link.text.split('Ext:')[-1].strip() 
        
    facultyIMG = None  # Initialize to None
    img_div = soup2.find('div', class_='img-thumbnail')
    if img_div:
        facultyIMG = 'http://isb.nu.edu.pk' + img_div.find('img')['src']
    
    isSupervisor = False
    p_element = soup2.findAll('p', style = 'line-height: 20px;margin-bottom: 0px;')
    if p_element:
        for i in range(len(p_element)):
            phd = p_element[i].find('label', style='font-size:9pt;')
            if phd:
                isSupervisor = phd.text.strip() == 'HEC Approved PhD Supervisor'
                

    ## Highest Education using Selenium
    max_retries = 3  # Set the maximum number of retries

    for retry in range(max_retries):
        try:
            driver.get(facultyLink)

            facultyEducation = None
            degree_name_span = driver.find_element(By.ID, 'DegreeName')
            if degree_name_span:
                education_items = degree_name_span.find_elements(By.TAG_NAME, 'li')
                if education_items:
                    facultyEducation = education_items[0].text.strip().split(',')[0]
            break  # Exit the loop if successful
        except Exception as e:
#             print(f"Error on attempt {retry + 1}: {e}")
            time.sleep(1)  # Add a delay before retrying


    ##############################
    
    try:
        id_.append(int(facultyId))
    except:
        id_.append(-1)
        
    name.append(facultyName)
    designation.append(facultyDesignation)
    phdSupervisor.append(isSupervisor)
    email.append(facultyEmail)
    dept.append(facultyDepartment)
    
    try:
        ext.append(int(facultyExtension))
    except:
        ext.append(-1)
        
    img.append(facultyIMG)
    highesteducation.append(facultyEducation)
    
# Close the WebDriver
driver.quit()

In [92]:
df = pd.DataFrame({'ID':id_,
                   'Name':name,
                   'Designation':designation,
                   'HEC Approved PHD Supervisor':phdSupervisor,
                   'Highest Education':highesteducation,
                   'Email':email,
                   'Department':dept,
                   'Extension':ext,
                   'ImageURL':img})

In [93]:
df

,ID,Name,Designation,HEC Approved PHD Supervisor,Highest Education,Email,Department,Extension,ImageURL
0,4553,Waseem Shahzad,Director & Professor,True,None,waseem.shahzad@nu.edu.pk,Computing,101,http://isb.nu.edu.pk/Images/Profile/CS/4553.jpg
1,3629,Kashif Munir,"Head, School of Computing & Professor",True,Ph.D (Computer Science) University of Innsbruc...,kashif.munir@nu.edu.pk,Computing,626,http://isb.nu.edu.pk/Images/Profile/CS/3629202...
2,4400,Hammad Majeed,HoD (Computer Science) & Professor,True,None,hammad.majeed@nu.edu.pk,Computing,672,http://isb.nu.edu.pk/Images/Profile/CS/4400.jpg
3,4551,Hasan Mujtaba Kiyani,HoD (Artificial Intelligence & Data Science) &...,True,Ph.D (Computer Science) National University of...,hasan.mujtaba@nu.edu.pk,Computing,508,http://isb.nu.edu.pk/Images/Profile/CS/4551.jpg
4,5099,Muhammad Asim,HoD (Cyber Security) & Professor,True,P.hD (Computer Science) Liverpool John Moore U...,muhammad.asim@nu.edu.pk,Computing,503,http://isb.nu.edu.pk/Images/Profile/CS/5099.jpg
...,...,...,...,...,...,...,...,...,...
198,5354,Sehrish Hassan Shigri,Lecturer,False,None,sehrish.hassan@nu.edu.pk,Science & Humanities,257,http://isb.nu.edu.pk/Images/Profile/SH/5354.jpg
199,5898,Sana Ilyas,Lecturer,False,None,sanaa.ilyas@nu.edu.pk,Science & Humanities,612,http://isb.nu.edu.pk/Images/user/avatar_2x.png
200,5902,Zunera Malik,Lecturer,False,None,zunaira.malik@nu.edu.pk,Science & Humanities,382,http://isb.nu.edu.pk/Images/user/avatar_2x.png
201,9341,Sadia Nauman,Lecturer,False,M.Phil (English) Government College University,sadia.nauman@nu.edu.pk,Science & Humanities,532,http://isb.nu.edu.pk/Images/Profile/SH/9341.jpg


In [73]:
df.to_csv('isb.csv')

In [94]:
df['Highest Education'].isnull().sum()

71

# Test for one faculty member

In [12]:
faculty = soup.findAll('div', class_='our-team')[0]
print(faculty.prettify())

<div class="our-team">
 <div class="pic">
  <img id="myImage4553" src="./fast_nu_isb_files/4553.jpg"/>
 </div>
 <div class="team-content">
  <h3 class="title">
   Waseem Shahzad
  </h3>
  <span class="post">
   Director &amp; Professor
  </span>
  <p>
   waseem.shahzad@nu.edu.pk
  </p>
  <br/>
 </div>
 <ul class="social">
  <li>
   <a href="http://isb.nu.edu.pk/Faculty/Details/4553" style="font-style: italic;" target="_blank">
    More...
   </a>
  </li>
 </ul>
</div>



In [ ]:
facultyId = faculty.find('img')['id'][-4:]
facultyId

In [ ]:
facultyName = faculty.find('h3', class_='title').text.strip()
facultyName

In [ ]:
facultyDesignation = faculty.find('span', class_='post').text.strip()
facultyDesignation

In [ ]:
facultyEmail = faculty.find('p').text
facultyEmail

In [ ]:
facultyDepartment = faculty.find_previous('div', class_='animated fadeInLeft').text
# pt2 = faculty.find_previous('div', class_='animated fadeInRight').text
# facultyDepartment = pt1 + ' ' + pt2
facultyDepartment

In [16]:
#Faculty link
facultyLink = faculty.find('ul', class_='social').find_next('a')['href']
facultyLink

'http://isb.nu.edu.pk/Faculty/Details/4553'

In [ ]:
#new soup object
re = requests.get(facultyLink)
#if re.status_code == '200':
soup2 = BeautifulSoup(re.content, 'html.parser')
#soup2.find('div', class_='profile-info')

In [ ]:
# Find the <span> element with the id "DegreeName"
edu = soup2.find('span', id='DegreeName').find('li').find('label')
print(edu)

# if degree_name_span:
#     # Find all <li> elements within the <span> element
#     education_items = degree_name_span.find_all('label')[0]
    
#     # Extract and print the education details
#     for item in education_items:
#         education_text = item.text
        
# education_text
#         print(education_text)
# else:
#     print("Education information not found.")


In [ ]:
facultyExtension = None
p_element = soup2.findAll('p', style = 'line-height: 20px;margin-bottom: 0px;')
if p_element:
    for i in range(len(p_element)):
        ext = p_element[i].find('span')
    facultyExtension = ext.text.split('Ext:')[-1].strip()
    
facultyExtension

In [ ]:
facultyIMG = None  # Initialize to None
img_div = soup2.find('div', class_='img-thumbnail')
if img_div:
    facultyIMG = 'http://isb.nu.edu.pk' + img_div.find('img')['src']
    
facultyIMG

In [ ]:
isSupervisor = False
p_element = soup2.findAll('p', style = 'line-height: 20px;margin-bottom: 0px;')
if p_element:
    for i in range(len(p_element)):
        phd = p_element[i].find('label', style='font-size:9pt;')
        if phd:
            isSupervisor = phd.text.strip() == 'HEC Approved PhD Supervisor'

isSupervisor
# soup2.findAll('p', style = 'line-height: 20px;margin-bottom: 0px;')[1].find('label', style='font-size:9pt;').text.strip()

In [ ]:
# ed = soup2.find('div', class_='form-group', id='EducationForHide')
# if ed:
#     span = ed.find_next('span', id='DegreeName')
#     if span:
#         e = span.find_next('li')
        
# e

In [97]:

chrome_path = "/Users/shahbanowaqar/Downloads/chromedriver_mac_arm64 (1)/chromedriver" 
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")

driver: WebDriver = webdriver.Chrome(service=Service(chrome_path), options=chrome_options)
driver.get('http://isb.nu.edu.pk/Faculty/Details/4553')

degree_name_span = driver.find_element(By.ID, 'DegreeName')
if degree_name_span:
    education_items = degree_name_span.find_elements(By.TAG_NAME, 'li')
    if education_items:
        facultyEducation = education_items[0].text.strip().split(',')[0]
else:
    facultyEducation = 'None'

# Close the WebDriver
driver.quit()

facultyEducation

'MA (English) National University of Modern Languages'